In [8]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

from sklearn.metrics import r2_score

In [9]:
data = pd.read_csv('C:/Users/kruna/Downloads/train.csv')

In [10]:
data

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

# Preprocessing

In [24]:



def encode_dates(df, column):
    df = df.copy()
    df[column] = pd.to_datetime(df[column])
    df[column + '_year'] = df[column].apply(lambda x: x.year)
    df[column + '_month'] = df[column].apply(lambda x: x.month)
    df[column + '_day'] = df[column].apply(lambda x: x.day)
    df = df.drop(column, axis=1)
    return df

def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [13]:
# Over here I am trying to build a model which would predict sales without taking into consideration which customer is buying a product.
# We would be dropping unnecessary columns like Row Id, Customer Id , Customer Name , Product Name, Country. ( we have droped Country because it has only one value.) 

In [25]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop unnecessary columns
    df = df.drop(['Row ID', 'Customer Name', 'Country', 'Product Name'], axis=1)
    
    # Drop customer-specific feature columns
    df = df.drop(['Order ID', 'Customer ID'], axis=1)
    
    # Extract date features
    df = encode_dates(df, column='Order Date')
    df = encode_dates(df, column='Ship Date')
    
    # One-hot encode categorical features
    for column in ['Ship Mode', 'Segment', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category']:
        df = onehot_encode(df, column=column)
    
    # Split df into X and y
    y = df['Sales']
    X = df.drop('Sales', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    return X_train, X_test, y_train, y_test

In [26]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [27]:
X_train

,Order Date_year,Order Date_month,Order Date_day,Ship Date_year,Ship Date_month,Ship Date_day,Ship Mode_First Class,Ship Mode_Same Day,Ship Mode_Second Class,Ship Mode_Standard Class,...,Sub-Category_Envelopes,Sub-Category_Fasteners,Sub-Category_Furnishings,Sub-Category_Labels,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables
0,1.153020,-0.351009,-0.142358,1.138089,-0.387963,0.315968,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
1,-0.626244,0.817035,-0.389593,-0.638920,0.792139,0.076408,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
2,1.153020,-1.811064,-1.131297,1.138089,0.202088,-1.121394,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,2.498785,-0.314653,-0.30333,-0.13789,-0.179443
3,-0.626244,1.109046,0.475728,-0.638920,1.087165,1.034650,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
4,1.153020,-0.643020,-0.884062,1.138089,0.497114,-0.881834,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,2.498785,-0.314653,-0.30333,-0.13789,-0.179443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6855,1.153020,0.817035,0.846580,1.138089,0.792139,1.513771,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,3.178101,-0.30333,-0.13789,-0.179443
6856,0.263388,-1.519053,-0.265975,0.249584,-1.568065,-0.163153,2.326202,-0.249166,-0.484918,-1.212556,...,-0.162742,-0.145387,3.064953,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
6857,0.263388,0.233013,-1.502149,0.249584,1.382191,-1.480735,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,2.498785,-0.314653,-0.30333,-0.13789,-0.179443
6858,-1.515875,1.109046,1.464667,-1.527425,-1.863091,-0.522493,-0.429885,-0.249166,2.062203,-1.212556,...,-0.162742,-0.145387,3.064953,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443


In [28]:
X_train.describe()

,Order Date_year,Order Date_month,Order Date_day,Ship Date_year,Ship Date_month,Ship Date_day,Ship Mode_First Class,Ship Mode_Same Day,Ship Mode_Second Class,Ship Mode_Standard Class,...,Sub-Category_Envelopes,Sub-Category_Fasteners,Sub-Category_Furnishings,Sub-Category_Labels,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables
count,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,...,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03,6.860000e+03
mean,8.941205e-14,-1.303187e-16,-1.898708e-16,-1.168806e-14,-1.873461e-16,1.009558e-16,-3.038548e-16,1.877911e-16,3.895410e-16,9.154323e-16,...,-2.156034e-16,-5.327209e-16,4.219171e-16,5.783760e-16,-2.546999e-16,-2.523249e-16,-1.618401e-16,1.237106e-16,-3.848881e-16,-9.371350e-17
std,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,...,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00,1.000073e+00
min,-1.515875e+00,-1.811064e+00,-1.873001e+00,-1.527425e+00,-1.863091e+00,-1.840076e+00,-4.298853e-01,-2.491664e-01,-4.849184e-01,-1.212556e+00,...,-1.627425e-01,-1.453865e-01,-3.262692e-01,-1.879240e-01,-1.086251e-01,-4.001944e-01,-3.146533e-01,-3.033297e-01,-1.378901e-01,-1.794431e-01
25%,-6.262437e-01,-9.350309e-01,-7.604448e-01,-6.389203e-01,-6.829885e-01,-7.620538e-01,-4.298853e-01,-2.491664e-01,-4.849184e-01,-1.212556e+00,...,-1.627425e-01,-1.453865e-01,-3.262692e-01,-1.879240e-01,-1.086251e-01,-4.001944e-01,-3.146533e-01,-3.033297e-01,-1.378901e-01,-1.794431e-01
50%,2.633881e-01,2.330128e-01,-1.874082e-02,2.495843e-01,2.020882e-01,-4.337232e-02,-4.298853e-01,-2.491664e-01,-4.849184e-01,8.247043e-01,...,-1.627425e-01,-1.453865e-01,-3.262692e-01,-1.879240e-01,-1.086251e-01,-4.001944e-01,-3.146533e-01,-3.033297e-01,-1.378901e-01,-1.794431e-01
75%,1.153020e+00,8.170347e-01,8.465805e-01,1.138089e+00,7.921394e-01,9.148696e-01,-4.298853e-01,-2.491664e-01,-4.849184e-01,8.247043e-01,...,-1.627425e-01,-1.453865e-01,-3.262692e-01,-1.879240e-01,-1.086251e-01,-4.001944e-01,-3.146533e-01,-3.033297e-01,-1.378901e-01,-1.794431e-01
max,1.153020e+00,1.401057e+00,1.835519e+00,2.026594e+00,1.382191e+00,1.753331e+00,2.326202e+00,4.013382e+00,2.062203e+00,8.247043e-01,...,6.144678e+00,6.878216e+00,3.064953e+00,5.321301e+00,9.205976e+00,2.498785e+00,3.178101e+00,3.296743e+00,7.252155e+00,5.572798e+00


In [29]:
y_train

8788    698.352
847     287.940
3397     25.920
6244    119.040
8198     15.552
         ...   
2895     35.880
7813     10.560
905      12.960
5192    397.600
235     617.976
Name: Sales, Length: 6860, dtype: float64

In [30]:
y_train.describe()

count     6860.000000
mean       234.030575
std        660.951936
min          0.444000
25%         17.340000
50%         54.514000
75%        212.940000
max      22638.480000
Name: Sales, dtype: float64

# Training

In [31]:


#A two hidden layers neural network is developed ( 1. input layer and 2. two dense layer)

inputs = tf.keras.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Dense(2048, activation='relu')(inputs)
x = tf.keras.layers.Dense(2048, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3102)]            0         
                                                                 
 dense_3 (Dense)             (None, 2048)              6354944   
                                                                 
 dense_4 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_5 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 10,553,345
Trainable params: 10,553,345
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

Epoch 1/100
172/172 [==============================] - 8s 45ms/step - loss: 434940.3125 - val_loss: 160971.2969 - lr: 0.0010
Epoch 2/100
172/172 [==============================] - 8s 45ms/step - loss: 285409.9375 - val_loss: 412977.9062 - lr: 0.0010
Epoch 3/100
172/172 [==============================] - 7s 43ms/step - loss: 163313.7812 - val_loss: 1226673.7500 - lr: 0.0010
Epoch 4/100
172/172 [==============================] - 8s 45ms/step - loss: 92059.9688 - val_loss: 3036100.7500 - lr: 0.0010
Epoch 5/100
172/172 [==============================] - 8s 45ms/step - loss: 69573.6641 - val_loss: 4232857.5000 - lr: 0.0010
Epoch 6/100
172/172 [==============================] - 8s 46ms/step - loss: 42895.5352 - val_loss: 3518061.7500 - lr: 0.0010


# Results

In [33]:

test_loss = model.evaluate(X_test, y_test, verbose=0)

print("Test Loss: {:.5f}".format(test_loss))



Test Loss: 208526.46875


In [37]:
y_pred = np.squeeze(model.predict(X_test))
test_r2 = r2_score(y_test, y_pred)



print("Test R^2 Score: {:.5f}".format(test_r2))

Test R^2 Score: 0.27988


# Conclusion
Here we conclude that the model is not functioning good even at 2048 nureons and thus we conclude that this model is not good and we need to develope more effecient model. 
According to the R^2 value it should be near 1.0. 